In [1]:
%endpoint https://query.wikidata.org/sparql
%prefix wd: <http://www.wikidata.org/entity/>
%prefix wds: <http://www.wikidata.org/entity/statement/>
%prefix wdv: <http://www.wikidata.org/value/>
%prefix wdt: <http://www.wikidata.org/prop/direct/>
%prefix wikibase: <http://wikiba.se/ontology#>
%prefix p: <http://www.wikidata.org/prop/>
%prefix ps: <http://www.wikidata.org/prop/statement/>
%prefix pq: <http://www.wikidata.org/prop/qualifier/>
%prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
%prefix bd: <http://www.bigdata.com/rdf#>

Endpoint set to: https://query.wikidata.org/sparql
Prefix set: wd: = <http://www.wikidata.org/entity/>
Prefix set: wds: = <http://www.wikidata.org/entity/statement/>
Prefix set: wdv: = <http://www.wikidata.org/value/>
Prefix set: wdt: = <http://www.wikidata.org/prop/direct/>
Prefix set: wikibase: = <http://wikiba.se/ontology#>
Prefix set: p: = <http://www.wikidata.org/prop/>
Prefix set: ps: = <http://www.wikidata.org/prop/statement/>
Prefix set: pq: = <http://www.wikidata.org/prop/qualifier/>
Prefix set: rdfs: = <http://www.w3.org/2000/01/rdf-schema#>
Prefix set: bd: = <http://www.bigdata.com/rdf#>

In [2]:
SELECT ?gene ?tax_id
WHERE
{
    ?gene wdt:P31 wd:Q7187 ; # ?gene instance of gene
          wdt:P593 "22758" ; # ?gene homologeneID is 22758
          wdt:P703 ?tax_id . # ?gene is found in NCBI taxonomy id ?tax_id
}

gene,tax_id
http://www.wikidata.org/entity/Q17709877,http://www.wikidata.org/entity/Q15978631
http://www.wikidata.org/entity/Q18247422,http://www.wikidata.org/entity/Q83310
http://www.wikidata.org/entity/Q24395030,http://www.wikidata.org/entity/Q184224
http://www.wikidata.org/entity/Q29674557,http://www.wikidata.org/entity/Q91703
http://www.wikidata.org/entity/Q29723729,http://www.wikidata.org/entity/Q130888
